In [60]:
import random
import numpy as np
import csv

class TableNode(object):
    def __init__(self, index):
        self.val = index
        self.buckets = {}


def genPara(n, r):
    """
    :param n: length of data vector
    :param r:
    :return: a, b
    """

    a = []
    for i in range(n):
        a.append(random.gauss(0, 1))
    b = random.uniform(0, r)

    return a, b


def gen_e2LSH_family(n, k, r):
    """
    :param n: length of data vector
    :param k: the number of hash function
    :param r: the width of bucket
    :return: a list of parameters (a, b)
    """
    result = []
    for i in range(k):
        result.append(genPara(n, r))

    return result


def gen_HashVals(e2LSH_family, v, r):
    """
    :param e2LSH_family: include k hash funcs(parameters)
    :param v: data vector
    :param r:the width of bucket
    :return hash values: a list
    """

    # hashVals include k values
    hashVals = []

    for hab in e2LSH_family:
        hashVal = (np.inner(hab[0], v) + hab[1]) // r
        hashVals.append(hashVal)

    return hashVals


def H2(hashVals, fpRand, k, C):
    """
    :param hashVals: k hash vals
    :param fpRand: ri', the random vals that used to generate fingerprint
    :param k, C: parameter
    :return: the fingerprint of (x1, x2, ..., xk), a int value
    """
    return int(sum([(hashVals[i] * fpRand[i]) for i in range(k)]) % C)


def e2LSH(dataSet, k, L, r, tableSize):
    """
    generate hash table
    * hash table: a list, [node1, node2, ... node_{tableSize - 1}]
    ** node: node.val = index; node.buckets = {}
    *** node.buckets: a dictionary, {fp:[v1, ..], ...}
    :param dataSet: a set of vector(list)
    :param k:the number of hash function
    :param L:
    :param r:
    :param tableSize:
    :return: 3 elements, hash table, hash functions, fpRand
    """

    hashTable = [TableNode(i) for i in range(tableSize)]

    #数据集列数
    n = len(dataSet[0])
    #数据集行数
    m = len(dataSet)

    C = pow(2, 32) - 5
    hashFuncs = []
    fpRand = [random.randint(-10, 10) for i in range(k)]

    for times in range(L):

        #得到k个hash 函数
        e2LSH_family = gen_e2LSH_family(n, k, r)

        # hashFuncs: [[h1, ...hk], [h1, ..hk], ..., [h1, ...hk]]
        # hashFuncs include L hash functions group, and each group contain k hash functions
        hashFuncs.append(e2LSH_family)

        #对于数据集的每一行
        for dataIndex in range(m):

            print("start****************************************************")
            print("dataIndex")
            print(dataIndex)
            # generate k hash values 得到k个hash值
            hashVals = gen_HashVals(e2LSH_family, dataSet[dataIndex], r)

            print("hashVals")
            print(hashVals)
            # generate fingerprint
            fp = H2(hashVals, fpRand, k, C)
            print("fp")
            print(fp)
            # generate index
            index = fp % tableSize
            print("index")
            print(index)
            # find the node of hash table
            node = hashTable[index]
            print("node.val")
            print(node.val)
            print("node.buckets")
            print(node.buckets)
            # node.buckets is a dictionary: {fp: vector_list}
            if fp in node.buckets:

                # bucket is vector list
                bucket = node.buckets[fp]

                # add the data index into bucket
                bucket.append(dataIndex)

            else:
                node.buckets[fp] = [dataIndex]
            print("node.buckets[fp]")
            print(node.buckets[fp])
                
        print("end-------------------------------------------------")
    return hashTable, hashFuncs, fpRand


def nn_search(dataSet, query, k, L, r, tableSize):
    """
    :param dataSet:
    :param query:
    :param k:
    :param L:
    :param r:
    :param tableSize:
    :return: the data index that similar with query
    """

    result = set()

    temp = e2LSH(dataSet, k, L, r, tableSize)
    C = pow(2, 32) - 5

    hashTable = temp[0]
    hashFuncGroups = temp[1]
    fpRand = temp[2]

    for hashFuncGroup in hashFuncGroups:

        # get the fingerprint of query
        queryFp = H2(gen_HashVals(hashFuncGroup, query, r), fpRand, k, C)

        # get the index of query in hash table
        queryIndex = queryFp % tableSize

        # get the bucket in the dictionary
        if queryFp in hashTable[queryIndex].buckets:
            result.update(hashTable[queryIndex].buckets[queryFp])

    return result


def readData(fileName):
    """read csv data"""

    dataSet = []
    with open(fileName, "r") as csvFile:
        reader = csv.reader(csvFile)
        for line in reader:
            dataSet.append([float(item) for item in line])

    return dataSet


def euclideanDistance(v1, v2):
    """get euclidean distance of 2 vectors"""

    v1, v2 = np.array(v1), np.array(v2)
    return np.sqrt(np.sum(np.square(v1 - v2)))

if __name__ == "__main__":

    dataSet = readData("./data/irisData.txt")
    
    hashTable,hashFuncs,fpRand= e2LSH(dataSet, k=3, L=4, r=1, tableSize=10)
     
    print("主程序")
    for i in range (10):
        node = hashTable[i]
        print("node.val")
        print(node.val)
        print("node.buckets")
        print(node.buckets)
         
         
    
#     query = [-2.7769, -5.6967, 5.9179, 0.37671, 1]
#     indexes = e2LSH.nn_search(dataSet, query, k=20, L=5, r=1, tableSize=20)
#     for index in indexes:
#         print(euclideanDistance(dataSet[index], query))

start****************************************************
dataIndex
0
hashVals
[4.0, -8.0, -4.0]
fp
4294967151
index
1
node.val
1
node.buckets
{}
node.buckets[fp]
[0]
start****************************************************
dataIndex
1
hashVals
[3.0, -7.0, -4.0]
fp
4294967167
index
7
node.val
7
node.buckets
{}
node.buckets[fp]
[1]
start****************************************************
dataIndex
2
hashVals
[3.0, -7.0, -4.0]
fp
4294967167
index
7
node.val
7
node.buckets
{4294967167: [1]}
node.buckets[fp]
[1, 2]
start****************************************************
dataIndex
3
hashVals
[4.0, -7.0, -4.0]
fp
4294967160
index
0
node.val
0
node.buckets
{}
node.buckets[fp]
[3]
start****************************************************
dataIndex
4
hashVals
[4.0, -8.0, -4.0]
fp
4294967151
index
1
node.val
1
node.buckets
{4294967151: [0]}
node.buckets[fp]
[0, 4]
start****************************************************
dataIndex
5
hashVals
[4.0, -9.0, -4.0]
fp
4294967142
index
2
node.val
2

fp
4294966986
index
6
node.val
6
node.buckets
{4294967186: [13], 4294967076: [53, 59, 67, 69, 89, 90, 92, 94], 4294967086: [62, 64, 79, 80, 81, 82], 4294966986: [102, 107, 120, 125, 130, 143]}
node.buckets[fp]
[102, 107, 120, 125, 130, 143, 144]
start****************************************************
dataIndex
145
hashVals
[7.0, -13.0, -12.0]
fp
4294967005
index
5
node.val
5
node.buckets
{4294967095: [57, 93, 98], 4294966995: [100, 104, 108, 124, 128, 129, 132, 136, 140], 4294967005: [110, 112, 115, 116, 137, 139, 141]}
node.buckets[fp]
[110, 112, 115, 116, 137, 139, 141, 145]
start****************************************************
dataIndex
146
hashVals
[6.0, -12.0, -12.0]
fp
4294967021
index
1
node.val
1
node.buckets
{4294967151: [0, 4, 7, 10, 16, 17, 19, 20, 21, 23, 24, 26, 27, 28, 31, 32, 33, 39, 40, 43, 44, 46, 48, 49], 4294967161: [36], 4294967051: [51, 56, 63, 85, 86], 4294967041: [70, 83], 4294967021: [123, 126]}
node.buckets[fp]
[123, 126, 146]
start***********************

{4294967095: [57, 93, 98], 4294966995: [100, 104, 108, 124, 128, 129, 132, 136, 140], 4294967005: [110, 112, 115, 116, 137, 139, 141, 145, 147, 148], 175: [52, 65, 75, 77], 135: [53, 55, 67, 69, 80, 81], 165: [58, 76], 145: [66]}
node.buckets[fp]
[66, 84]
start****************************************************
dataIndex
85
hashVals
[7.0, 14.0, 8.0]
fp
157
index
7
node.val
7
node.buckets
{4294967167: [1, 2, 8, 9, 12, 34, 37, 38, 41, 45], 4294967177: [35], 4294967067: [55, 61, 68, 71, 73, 74, 78, 87, 88, 96, 97, 99], 4294966977: [105, 122, 135], 4294966967: [118], 137: [1, 2, 6, 11, 24, 29, 45, 47], 127: [3, 9, 12, 30, 34, 37], 177: [50], 117: [57], 157: [61, 70], 147: [63, 71, 78, 82]}
node.buckets[fp]
[61, 70, 85]
start****************************************************
dataIndex
86
hashVals
[7.0, 16.0, 8.0]
fp
175
index
5
node.val
5
node.buckets
{4294967095: [57, 93, 98], 4294966995: [100, 104, 108, 124, 128, 129, 132, 136, 140], 4294967005: [110, 112, 115, 116, 137, 139, 141, 145,

node.buckets
{4294967039: [106], 149: [0, 4, 17, 19, 21, 27, 32, 40, 43, 44, 46, 48], 139: [28, 39], 159: [100, 116, 123, 126, 127], 189: [117], 169: [136]}
node.buckets[fp]
[100, 116, 123, 126, 127, 137]
start****************************************************
dataIndex
138
hashVals
[8.0, 14.0, 8.0]
fp
150
index
0
node.val
0
node.buckets
{4294967160: [3, 6, 11, 29, 30, 42, 47], 4294967170: [22], 4294967060: [66, 84, 91, 95], 4294966970: [109, 131], 4294967030: [113, 119], 4294966960: [117], 170: [14], 140: [22]}
node.buckets[fp]
[138]
start****************************************************
dataIndex
139
hashVals
[8.0, 17.0, 9.0]
fp
187
index
7
node.val
7
node.buckets
{4294967167: [1, 2, 8, 9, 12, 34, 37, 38, 41, 45], 4294967177: [35], 4294967067: [55, 61, 68, 71, 73, 74, 78, 87, 88, 96, 97, 99], 4294966977: [105, 122, 135], 4294966967: [118], 137: [1, 2, 6, 11, 24, 29, 45, 47, 119, 134], 127: [3, 9, 12, 30, 34, 37], 177: [50, 102, 125], 117: [57], 157: [61, 70, 85, 114], 147: [63, 

node.buckets
{4294967160: [3, 6, 11, 29, 30, 42, 47], 4294967170: [22], 4294967060: [66, 84, 91, 95], 4294966970: [109, 131], 4294967030: [113, 119], 4294966960: [117], 170: [14], 140: [22], 150: [138], 4294967110: [50, 52], 4294967120: [51], 4294967140: [53]}
node.buckets[fp]
[51, 54]
start****************************************************
dataIndex
55
hashVals
[-5.0, -4.0, -16.0]
fp
4294967130
index
0
node.val
0
node.buckets
{4294967160: [3, 6, 11, 29, 30, 42, 47], 4294967170: [22], 4294967060: [66, 84, 91, 95], 4294966970: [109, 131], 4294967030: [113, 119], 4294966960: [117], 170: [14], 140: [22], 150: [138], 4294967110: [50, 52], 4294967120: [51, 54], 4294967140: [53]}
node.buckets[fp]
[55]
start****************************************************
dataIndex
56
hashVals
[-5.0, -4.0, -17.0]
fp
4294967120
index
0
node.val
0
node.buckets
{4294967160: [3, 6, 11, 29, 30, 42, 47], 4294967170: [22], 4294967060: [66, 84, 91, 95], 4294966970: [109, 131], 4294967030: [113, 119], 4294966960

dataIndex
134
hashVals
[-6.0, -4.0, -19.0]
fp
4294967107
index
7
node.val
7
node.buckets
{4294967167: [1, 2, 8, 9, 12, 34, 37, 38, 41, 45], 4294967177: [35, 18, 20], 4294967067: [55, 61, 68, 71, 73, 74, 78, 87, 88, 96, 97, 99], 4294966977: [105, 122, 135], 4294966967: [118], 137: [1, 2, 6, 11, 24, 29, 45, 47, 119, 134], 127: [3, 9, 12, 30, 34, 37], 177: [50, 102, 125], 117: [57], 157: [61, 70, 85, 114, 149], 147: [63, 71, 78, 82, 91, 101, 113, 142], 167: [107, 129], 187: [109, 120, 139, 141], 4294967187: [5, 14, 15, 16, 21, 23, 25, 26, 31, 36], 4294967207: [41], 4294967197: [43, 45], 4294967147: [106], 4294967127: [119], 4294967117: [133]}
node.buckets[fp]
[134]
start****************************************************
dataIndex
135
hashVals
[-7.0, -6.0, -21.0]
fp
4294967076
index
6
node.val
6
node.buckets
{4294967186: [13], 4294967076: [53, 59, 67, 69, 89, 90, 92, 94, 130], 4294967086: [62, 64, 79, 80, 81, 82, 102, 109], 4294966986: [102, 107, 120, 125, 130, 143, 144], 146: [7, 23, 26

node.buckets
{4294967014: [101, 111, 114, 127, 133, 138, 142, 149], 4294967004: [103, 134], 124: [41], 144: [62, 73, 83], 154: [92], 4294967194: [8, 38]}
node.buckets[fp]
[58]
start****************************************************
dataIndex
59
hashVals
[6.0, -1.0, -4.0]
fp
4294967200
index
0
node.val
0
node.buckets
{4294967160: [3, 6, 11, 29, 30, 42, 47], 4294967170: [22], 4294967060: [66, 84, 91, 95], 4294966970: [109, 131], 4294967030: [113, 119], 4294966960: [117], 170: [14], 140: [22], 150: [138], 4294967110: [50, 52, 72, 76, 86], 4294967120: [51, 54, 56, 58, 63, 65, 73, 74, 75, 83, 87, 91], 4294967140: [53, 61, 89, 92, 94], 4294967130: [55, 62, 66, 70, 71, 78, 85, 97], 4294967200: [55]}
node.buckets[fp]
[55, 59]
start****************************************************
dataIndex
60
hashVals
[4.0, -1.0, -4.0]
fp
4294967214
index
4
node.val
4
node.buckets
{4294967014: [101, 111, 114, 127, 133, 138, 142, 149], 4294967004: [103, 134], 124: [41], 144: [62, 73, 83], 154: [92], 429496

[5.0, -1.0, -4.0]
fp
4294967207
index
7
node.val
7
node.buckets
{4294967167: [1, 2, 8, 9, 12, 34, 37, 38, 41, 45, 51, 86], 4294967177: [35, 18, 20], 4294967067: [55, 61, 68, 71, 73, 74, 78, 87, 88, 96, 97, 99], 4294966977: [105, 122, 135], 4294966967: [118], 137: [1, 2, 6, 11, 24, 29, 45, 47, 119, 134], 127: [3, 9, 12, 30, 34, 37], 177: [50, 102, 125], 117: [57], 157: [61, 70, 85, 114, 149], 147: [63, 71, 78, 82, 91, 101, 113, 142], 167: [107, 129], 187: [109, 120, 139, 141], 4294967187: [5, 14, 15, 16, 21, 23, 25, 26, 31, 36], 4294967207: [41, 53, 62, 69, 80, 81, 93, 98, 101, 121], 4294967197: [43, 45, 103, 108, 111, 116, 123, 126], 4294967147: [106], 4294967127: [119, 149], 4294967117: [133], 4294967107: [134], 4294967157: [50, 52], 4294967217: [57]}
node.buckets[fp]
[41, 53, 62, 69, 80, 81, 93, 98, 101, 121, 127]
start****************************************************
dataIndex
128
hashVals
[5.0, -1.0, -5.0]
fp
4294967197
index
7
node.val
7
node.buckets
{4294967167: [1, 2, 8, 9, 1